In [1]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

/home/sambit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sambit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sambit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sambit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [9]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import tensorflow_datasets as tfds
from tensorflow.data import  TFRecordDataset
from glob import glob
from tensorflow_datasets.core import DatasetBuilder

import os

tf.enable_eager_execution()
assert tf.executing_eagerly()

def parse_fn(example):
    example_fmt = {
        'image': tf.io.FixedLenFeature((), tf.string, ""),
        'label': tf.io.FixedLenFeature((), tf.int64, -1)
    }
    parsed = tf.parse_single_example(example, example_fmt)
    image = tf.io.decode_image(parsed['image'])
    image = tf.reshape(image, [32, 32, 3])
    return image, parsed['label']

def make_dataset(path):
    dataset = tf.data.TFRecordDataset(path)
    dataset = dataset.map(parse_fn)
    return dataset

cifar10_bldr = tfds.builder("cifar10")
cifar10_info = cifar10_bldr.info

name = cifar10_info.name
vrsn = '{}'.format(cifar10_info.version)

len_train, len_test = cifar10_info.splits['train'].num_examples, cifar10_info.splits['test'].num_examples

cifar10_bldr.download_and_prepare()

train_path = os.path.join('/home/sambit/tensorflow_datasets/', name, vrsn, '*train.tfrecord*')
test_path  = os.path.join('/home/sambit/tensorflow_datasets/', name, vrsn, '*test.tfrecord*')

train_fn = glob(train_path)
test_fn  = glob(test_path)

train_tfr, test_tfr = make_dataset(train_fn), make_dataset(test_fn)

train_mean = (0.4914, 0.4822, 0.4465)
train_std  = (0.2023, 0.1994, 0.2010)

data_aug  = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)
normalize = lambda x, y: ((tf.cast(x, tf.float32) - train_mean) / train_std, y)
pad4      = lambda x, y: (tf.pad(x, [[4, 4], [4, 4], [0, 0]], mode='reflect'), y)

train_ds = train_tfr.map(pad4).map(normalize).map(data_aug)
test_ds  = test_tfr.map(normalize)

val = train_tfr.take(10)

sumx = 0
for aaa in train_ds:
    sumx += 1
    
print(sumx)

50000
